In [1]:
import numpy as np
import math
from math import pi
import sympy as sp
from sympy import sin,cos
import matplotlib.pyplot as plt
import copy

from my_robot_dyne import Quaternion,Link,Robot,jacobians,gravity_torques,calc_b,calc_c

In [10]:
q1 = sp.Symbol('q1')
q2 = sp.Symbol('q2')
q3 = sp.Symbol('q3')

dq1 = sp.Symbol('dq1')
dq2 = sp.Symbol('dq2')
dq3 = sp.Symbol('dq3')

ddq1 = sp.Symbol('ddq1')
ddq2 = sp.Symbol('ddq2')
ddq3 = sp.Symbol('ddq3')

l1 = sp.Symbol('l1')
l2 = sp.Symbol('l2')
l3 = sp.Symbol('l3')

a1 = sp.Symbol('a1')
a2 = sp.Symbol('a2')
a3 = sp.Symbol('a3')

g = -9.81

In [3]:
links = [Link(pi/2,0,q1,0.05,0), Link(0, 0.1, q2, 0,0.05),Link(0,0.1,q3,0,0.05)]
Mp = [0,0,0]
K = [1,1,1]
ml = [0.01,0.12,0.08]
mm = [0.034,0.034,0.034]
v0 = np.array([0, 0, 0])
n = len(links)  # number of links
Il = {}
Im = {}
Il['1'] = sp.Matrix([
    [2.632e-5, 0.000e+0, 0.000e+0],
    [0.000e+0, 3.848e-5, 3.4e-7],
    [0.000e+0, 3.4e-7, 2.176e-5]
])
Il['2'] = sp.Matrix([
    [3.335e-5, 7.64e-6, 1.7e-6],
    [7.64e-6, 1.8732e-4, 1.1e-7],
    [1.7e-6, 1.1e-7, 1.9302e-4]
])
Il['3'] = sp.Matrix([
    [2.046e-5, 0.000e+0, -3.4e-7],
    [0.000e+0, 9.580e-5, 0.000e+0],
    [-3.4e-7, 0.000e+0, 9.269e-5]
])
Im['1'] = sp.Matrix([
    [3.20e-6, 0, 0],
    [0, 2.74e-6, 3.0e-8],
    [0, 3.0e-8, 3.70e-6]
])
Im['2'] = sp.Matrix([
    [3.20e-6, 0, 0],
    [0, 2.74e-6, 3.0e-8],
    [0, 3.0e-8, 3.70e-6]
])
Im['3'] = sp.Matrix([
    [3.20e-6, 0, 0],
    [0, 2.74e-6, 3.0e-8],
    [0, 3.0e-8, 3.70e-6]
])

In [4]:
JPl, Jol, JPm, Jom, R_dict, pe_link, pe_motor = jacobians(links, Mp, K, v0, g)

In [11]:
tau_g,G = gravity_torques(links, Mp, ml, mm, v0, g)
G.subs({q1:0,q2:0,q3:0})

Matrix([
[       0],
[0.209934],
[ 0.03924]])

In [7]:
B = calc_b(n, ml, JPl, Jol, R_dict, Il, mm, JPm, Jom, Im)
B

Matrix([
[3.77475828372553e-22*sin(q2) + 3.76681082243033e-37*sin(2*q2) - 1.52947417914311e-22*sin(q2 + q3) + 1.88737914186277e-23*sin(-2*q1 + q2 + q3) + 1.88737914186277e-23*sin(2*q1 + q2 + q3) + 3.3881317890172e-20*cos(2*q1) + 3.10862446895044e-23*cos(q2) + 0.00072*cos(2*q2) + 0.0004*cos(q3) + 1.29702672213572e-21*cos(2*q3) - 1.6940658945086e-21*cos(2*(q1 - q2)) - 1.6940658945086e-21*cos(2*(q1 + q2)) - 6.48513361067859e-22*cos(2*(q1 - q3)) - 6.48513361067859e-22*cos(2*(q1 + q3)) + 1.32787830481752e-23*cos(q2 + q3) + 0.0001*cos(2*(q2 + q3)) + 0.0004*cos(2*q2 + q3) + 3.24256680533929e-22*cos(2*(-q1 + q2 + q3)) + 3.2425668053393e-22*cos(2*(q1 - q2 + q3)) + 3.2425668053393e-22*cos(2*(q1 + q2 - q3)) + 3.24256680533931e-22*cos(2*(q1 + q2 + q3)) + 0.00113857,                                                                                                                                                                                                                                            

In [8]:
C = calc_c(B,n)
C

[[dq1*(-3.3881317890172e-20*sin(2*q1) + 1.6940658945086e-21*sin(2*(q1 - q2)) + 1.6940658945086e-21*sin(2*(q1 + q2)) + 6.48513361067859e-22*sin(2*(q1 - q3)) + 6.48513361067859e-22*sin(2*(q1 + q3)) + 3.24256680533929e-22*sin(2*(-q1 + q2 + q3)) - 3.2425668053393e-22*sin(2*(q1 - q2 + q3)) - 3.2425668053393e-22*sin(2*(q1 + q2 - q3)) - 3.24256680533931e-22*sin(2*(q1 + q2 + q3)) - 1.88737914186277e-23*cos(-2*q1 + q2 + q3) + 1.88737914186277e-23*cos(2*q1 + q2 + q3)) + dq2*(-1.55431223447522e-23*sin(q2) - 0.00072*sin(2*q2) - 1.6940658945086e-21*sin(2*(q1 - q2)) + 1.6940658945086e-21*sin(2*(q1 + q2)) - 6.63939152408758e-24*sin(q2 + q3) - 0.0001*sin(2*(q2 + q3)) - 0.0004*sin(2*q2 + q3) - 3.24256680533929e-22*sin(2*(-q1 + q2 + q3)) + 3.2425668053393e-22*sin(2*(q1 - q2 + q3)) - 3.2425668053393e-22*sin(2*(q1 + q2 - q3)) - 3.24256680533931e-22*sin(2*(q1 + q2 + q3)) + 1.88737914186277e-22*cos(q2) + 3.76681082243033e-37*cos(2*q2) - 7.64737089571556e-23*cos(q2 + q3) + 9.43689570931383e-24*cos(-2*q1 + q2

In [12]:
ddq = sp.Matrix([ddq1, ddq2, ddq3])
dq = sp.Matrix([dq1, dq2, dq3])
tau = B*ddq+sp.Matrix(C)*dq+G
joint_torques = tau.subs({ddq1:pi/2,ddq2:pi/2,ddq3:pi/2,dq1:pi/2,dq2:pi/2,dq3:pi/2,q1:0,q2:0,q3:0})
joint_torques.subs({a1:0,l1:0,a2:0.1,l2:0.05,a3:0.1,l3:0.05})

Matrix([
[0.00494497679249321],
[   0.21578146211409],
[ 0.0409508642352552]])